In [67]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import json
import numpy as np
import pandas as pd
import datetime
from pandas import json_normalize

#this JSON file is from the service account user email we created in the Google Drive 
#remember to give permissions to the service account Email
key_file_location = 'C:/Users/Ops/Documents/Python Scripts/aircallautomation.json'
#scope is standard
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_location, scope)
client = gspread.authorize(creds)
#aircall API data
API_KEY =  '<AIRCALL_API_KEY>'
API_SECRET ='AIRCALL_API_KEY_SECRET'
API_ENDPOINT = 'https://api.aircall.io/v1/calls'
#defines data call from Aircall API using json format (data in API is called "calls")
def get_call_data(api_key, api_secret, start_date=None, page=1):
    headers = {'content-type': 'application/json'}
    params = {'page': page, 'direction': 'inbound,outbound', 'answered': 'true'}
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    start_date = yesterday.strftime('%Y-%m-%d') if start_date is None else start_date
    if start_date:
        params['start_date'] = start_date
    try:
        response = requests.get(API_ENDPOINT, headers=headers, auth=(api_key, api_secret), params=params)
        response_data = json.loads(response.text)
    except Exception as e:
        print(f"Error getting call data from API: {e}")
        return pd.DataFrame()
    if 'calls' not in response_data:
        print(f"Invalid API response: {response.text}")
        return pd.DataFrame()
    
    calls_data = response_data['calls']
    df = pd.DataFrame(calls_data)
    df.fillna(value='', inplace=True)
    #edit the columns that return dictionaries
    df_number = json_normalize(df['number'])
    df_number.columns = [f"number.{col}" for col in df_number.columns]
    df_tag = json_normalize(df['tags'])
    df_tag.columns = [f"tag.{col}" for col in df_tag.columns]
    df_user = json_normalize(df['user'])
    df_user.columns = [f"user.{col}" for col in df_user.columns]
    
    # Concatenate the normalized columns to the original DataFrame
    df = pd.concat([df, df_number, df_tag, df_user], axis=1)
    df.drop(['number', 'tags', 'user'], axis=1, inplace=True)
    
    df = df.astype(str)
    return df

In [ ]:
#this has to do with google sheets API connection I(from the defined drive, folder and file API)
def update_google_sheet(sheet, data, worksheet_name):
    worksheet = sheet.worksheet(worksheet_name)
    worksheet.clear()
    data = data.replace([np.inf, -np.inf], np.nan)
    data.fillna(value='N/A', inplace=True)
    data = data.applymap(lambda x: None if x in ([], [np.nan], [None]) else x)
    data_list = data.values.tolist()
    worksheet.update([data.columns.values.tolist()] + data.values.tolist())
    print(f"{len(data_list)} rows updated successfully in worksheet '{worksheet_name}'")


df = pd.DataFrame()
page = 1
while True:
    new_data = get_call_data(API_KEY, API_SECRET, page=page)
    if new_data.empty:
        break
    df = pd.concat([df, new_data], ignore_index=True)
    page += 1

# Update Google Sheet with call data
df = df.applymap(str)
df = df.fillna('')
sheet = client.open('AirCall_WebHook')
sheet_name = 'Data'
update_google_sheet(sheet, df, sheet_name)
print(df)